In [1]:
from polars_ds import StrExt, NumExt
import polars as pl
import numpy as np 

In [2]:
size = 100_000
df = pl.DataFrame({
    "a": 2 + np.sin(np.array(range(size))),
    "1024":pl.Series([1024] * size, dtype = pl.UInt32),
    "exp": pl.Series([x % 100 for x in range(size)], dtype =  pl.UInt32),
    "exp2": pl.Series(range(size), dtype =  pl.UInt32)
})

In [3]:
from polars.testing import assert_frame_equal

f1 = df.select(pl.col("a").pow(pl.lit(100, pl.Int32)))
f2 = df.select(pl.col("a").num_ext.powi(100))

assert_frame_equal(f1,f2)

In [10]:
%timeit df.select(pl.col("a").pow(pl.lit(100_000, pl.Int32)))
%timeit df.select(pl.col("a").num_ext.powi(100_000)) # case 1

1.14 ms ± 4.83 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
556 µs ± 3.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
f1 = df.select(pl.col("a").pow(pl.col("1024")))
f2 =  df.select(pl.col("a").num_ext.powi(pl.col("1024")))
assert_frame_equal(f1,f2)

In [12]:
%timeit df.select(pl.col("a").pow(pl.col("1024")))
%timeit df.select(pl.col("a").num_ext.powi(pl.col("1024")))

1.32 ms ± 5.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
616 µs ± 4.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
f1 = df.select(pl.col("a").pow(pl.col("exp2")))
f2 =  df.select(pl.col("a").num_ext.powi(pl.col("exp2")))
assert_frame_equal(f1,f2)

In [15]:
f1 = df.select(pl.col("a").pow(pl.col("exp2")))
f2 =  df.select(pl.col("a").num_ext.powi(pl.col("exp2"))) 
assert_frame_equal(f1,f2)

In [16]:
%timeit df.select(pl.col("a").pow(pl.col("exp2"))) # case 3
%timeit df.select(pl.col("a").num_ext.powi(pl.col("exp2")))

1.22 ms ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
3.82 ms ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
df.select(pl.col("a").num_ext.powi(pl.col("exp")))

a
f64
1.0
2.841471
8.464012
9.81574
2.388694
1.222958
25.945141
934.814731
6377.107152


In [ ]:
df.select(pl.col("a").num_ext.powi(pl.col("exp")))

In [ ]:
# size = 100_000
# df = pl.DataFrame({
#     "a": np.sin(np.array(range(size))),
#     # "b": np.sin(np.array(range(size))),
#     # "c": np.sin(np.array(range(size))),
#     # "d": np.sin(np.array(range(size))),
#     # "e": np.sin(np.array(range(size))),
#     "1024":pl.Series([1024] * size, dtype = pl.Int32),
#     "exp": pl.Series(range(size), dtype = pl.Int32)
# })
# # npa = df["a"].to_numpy()
# # npb = df["b"].to_numpy()
# # npc = df["c"].to_numpy()
# # npd = df["d"].to_numpy()
# # npe = df["e"].to_numpy()

In [ ]:
%%timeit
np.fft.rfft(npa).real

In [ ]:
%%timeit
df.select(
    pl.col("a").num_ext.fft().struct.field("re"),
)

In [ ]:
%%timeit
df.select(
    pl.col("a").num_ext.fft(),
     pl.col("b").num_ext.fft(),
     pl.col("c").num_ext.fft(),
     pl.col("d").num_ext.fft(),
     pl.col("e").num_ext.fft()
)

In [ ]:
%%timeit 
_ = np.fft.rfft(npa)
_ = np.fft.rfft(npb)
_ = np.fft.rfft(npc)
_ = np.fft.rfft(npd)
_ = np.fft.rfft(npe)

In [3]:
df = pl.DataFrame({
    "dummy": ["a"] * 50_000 + ["b"] * 50_000,
    "a": range(100_000),
    "b": range(-100_000, 0),
    "y": pl.Series(range(100_000, 200_000)) + 0.5
})

df.head()

dummy,a,b,y
str,i64,i64,f64
"""a""",0,-100000,100000.5
"""a""",1,-99999,100001.5
"""a""",2,-99998,100002.5
"""a""",3,-99997,100003.5
"""a""",4,-99996,100004.5


In [4]:
df.group_by("dummy").agg(
    pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b"), add_bias=False)
)

dummy,list_float
str,list[f64]
"""a""","[2.000005, -1.000005]"
"""b""","[2.000005, -1.000005]"


In [ ]:
df.select(
    pl.col("a").num_ext.cond_entropy(pl.col("dummy"))
)

In [ ]:
df.group_by("dummy").agg(
    pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b"), add_bias = True)
)

In [6]:
%timeit df.select(pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b")))

725 µs ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
df.select(
    pl.col("y").num_ext.lstsq(pl.col("a"), pl.col("b")).over("dummy")
)

In [ ]:
df.select(
    pl.col("a").str_ext.levenshtein_dist("b")
)